<a href="https://colab.research.google.com/github/financieras/campus/blob/main/pilas/pilas22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmo Deep Moder Times

In [1]:
# FUNCIONES
def sa(a,b):
    if len(a) > 1: a[0],a[1] = a[1],a[0]
    return a,b
def sb(a,b):
    if len(b) > 1: b[0],b[1] = b[1],b[0]
    return a,b
def ss(a,b):
    sa(a,b)
    sb(a,b)
    return a,b
def pa(a,b):
    if len(b) > 0:
        a.insert(0, b[0])
        b.pop(0)
    return a,b
def pb(a,b):
    if len(a) > 0:
        b.insert(0, a[0])
        a.pop(0)
    return a,b
def ra(a,b):
    if len(a) > 1: a.append(a.pop(0))
    return a,b
def rb(a,b):
    if len(b) > 1: b.append(b.pop(0))
    return a,b
def rr(a,b):
    ra(a,b)
    rb(a,b)
    return a,b
def rra(a,b):
    if len(a) > 1: a.insert(0, a.pop())
    return a,b
def rrb(a,b):
    if len(b) > 1: b.insert(0, b.pop())
    return a,b
def rrr(a,b):
    rra(a,b)
    rrb(a,b)
    return a,b

In [2]:
# Generación de la pila A
from random import sample, seed
seed()

def generaPilaA(n):
    return sample(range(1, n+1), n)

In [3]:
# Llevamos de la pila A hacia la pila B los elementos de la LIS

def rotateLIS(lis):
    global a
    global b
    global contador
    for vlis in lis:
        if a.index(vlis) < len(a)/2:
            for i in range(a.index(vlis)):
                ra(a,b); contador += 1
        else:
            for i in range(len(a)- a.index(vlis)):
                rra(a,b); contador += 1
        pb(a,b); contador += 1

In [4]:
# PASOS NECESARIOS PARA COLOCAR CADA ELEMENTO DE A EN SU SITIO EN B
# PASOS NECESARIOS = Pasos necesarios A + Pasos necesarios B

# Pasos necesarios para situarse como el primer elemento de la pila A

def necesariosA(a, b):   # rellena el array pasosA con todos los pasos necesarios para la pila A
    for i in a:
        if a.index(i) < len(a)/2:
            pasosA.append(a.index(i))
        else:
            pasosA.append(-(len(a)- a.index(i)))

def necesariosB(a,b):   # rellena el array pasosB con todos los pasos necesarios para la pila B
    for v in a:
        try:
            objetivo_primero = max([x for x in b if x<v])
        except:
            objetivo_primero = max(b)   # objetivo_primero calcula el número que se ha de poner en la primera posición de B

        if b.index(objetivo_primero) < len(b)/2:
            pasosB.append(b.index(objetivo_primero))
        else:
            pasosB.append(-(len(b)- b.index(objetivo_primero)))

def totaliza(a,b):   # totalizar pasos
    global total
    for i in range(len(pasosA)):
        if pasosA[i] * pasosB[i] < 0:   # si son de distinto signo
            total.append(abs(pasosA[i]) + abs(pasosB[i]))
        else:
            total.append(max(abs(pasosA[i]), abs(pasosB[i])))

def calculaIndexPasosMinimos():
    global pasosA
    global pasosB
    global total
    pasosA = []   # [0,1,2, ...,41,-41,... , -2,-1]  vector donde cada index está asociado con el valor del mismo index en A
    pasosB = []   # [-3,2,-5, ..., -5,0]   estos dos arrays se han de recalcular cada vez que realmente se mueva algún elemento de A a B
    total = []
    necesariosA(a, b)
    necesariosB(a, b)
    totaliza(a,b)
    if len(b)==2:
        print("pasosA:", pasosA)
        print("pasosB:", pasosB)
        print("total:", total)
    return total.index(min(total))   # retorna el índice del elemento de la pila A que menos pasos totales necesita

In [5]:
def giraA(a, b, indice, pasos):
    global contador
    for i in range(abs(pasos)):
        if pasos > 0:
            ra(a,b); contador += 1
        elif pasos < 0:
            rra(a,b); contador += 1

def giraB(a, b, indice, pasos):
    global contador
    for i in range(abs(pasos)):
        if pasos > 0:
            rb(a,b); contador += 1
        elif pasos < 0:
            rrb(a,b); contador += 1

# girando pilas A y B
def giraPilas(a,b,indice):   # indice es el index del valor en la pila A que deseamos poner el primero
    global contador
    if pasosA[indice] * pasosB[indice] > 0:   # Existe sinergia, nos podemos ahorrar pasos
        pasos_comunes = min(abs(pasosA[indice]), abs(pasosB[indice]))
        for i in range(pasos_comunes):
            if pasosA[indice] > 0:   # si el signo de ambos es positivo, ya que ambos tienen el mismo signo
                rr(a,b); contador += 1
            elif pasosA[indice] < 0: # si el signo de ambos es negativo
                rrr(a,b); contador += 1
        exceso_pasosA = abs(pasosA[indice]) - pasos_comunes
        exceso_pasosB = abs(pasosB[indice]) - pasos_comunes
        giraA(a, b, indice, ((pasosA[indice] > 0) - (pasosA[indice] < 0)) * exceso_pasosA)    # (a > 0) - (a < 0) da el signo de a
        giraB(a, b, indice, ((pasosB[indice] > 0) - (pasosB[indice] < 0)) * exceso_pasosB)   # Python no tiene función sign
    else:   # No existe sinergia
        giraA(a ,b, indice, pasosA[indice])   # gira A
        giraB(a, b, indice, pasosB[indice])   # gira B

In [6]:
def situarMax_en_B():
    indice = b.index(max(b))
    if indice < len(a)/2:
        pasos = indice
    else:
        pasos = -(len(b)- indice)
    giraB(a, b, indice, pasos)

def subirTodo_a_A():
    global contador
    for _ in range(len(b)):
        pa(a,b); contador += 1

In [9]:
if __name__ == "__main__":    
    n = 100   # número de elementos de la pila
    a = generaPilaA(n)
    a = [161, 97, 226, 102, 11, 288, 148, 328, 154, 463, 73, 323, 284, 67, 34, 370, 228, 70, 199, 130, 424, 307, 245, 185, 496, 303, 119, 327, 457, 141, 24, 348, 406, 302, 349, 1, 80, 72, 319, 257, 285, 494, 419, 150, 308, 256, 135, 44, 412, 369, 206, 113, 10, 497, 413, 467, 136, 191, 180, 265, 126, 198, 27, 417, 125, 452, 346, 83, 280, 153, 326, 320, 232, 151, 49, 380, 474, 294, 21, 495, 461, 272, 414, 335, 482, 317, 120, 442, 234, 71, 358, 112, 210, 217, 405, 227, 357, 394, 281, 146, 485, 500, 75, 322, 404, 209, 408, 124, 350, 183, 50, 26, 332, 243, 375, 255, 143, 251, 337, 443, 204, 46, 374, 157, 268, 205, 382, 439, 305, 409, 220, 115, 339, 53, 498, 77, 6, 347, 277, 435, 96, 122, 388, 448, 78, 162, 446, 138, 127, 287, 137, 253, 433, 221, 18, 366, 459, 487, 84, 114, 196, 192, 312, 95, 468, 14, 315, 63, 23, 279, 297, 186, 411, 429, 269, 460, 479, 343, 94, 163, 74, 133, 398, 43, 438, 89, 5, 445, 45, 436, 156, 464, 383, 81, 129, 455, 418, 76, 355, 12, 484, 85, 353, 289, 68, 195, 132, 271, 361, 178, 51, 175, 62, 333, 431, 207, 310, 211, 331, 402, 218, 231, 407, 147, 389, 17, 158, 363, 352, 336, 456, 41, 92, 330, 140, 103, 311, 434, 465, 499, 116, 219, 152, 167, 16, 444, 466, 79, 261, 426, 313, 252, 437, 291, 37, 282, 318, 475, 483, 242, 321, 179, 365, 401, 381, 342, 110, 299, 222, 428, 184, 57, 32, 42, 202, 395, 316, 254, 194, 213, 169, 458, 376, 410, 469, 306, 190, 427, 490, 491, 216, 4, 493, 35, 107, 364, 275, 400, 58, 244, 134, 40, 33, 338, 159, 86, 432, 241, 168, 98, 176, 344, 203, 7, 48, 54, 329, 201, 422, 15, 415, 260, 13, 38, 423, 373, 188, 486, 384, 351, 229, 36, 66, 283, 387, 447, 421, 233, 197, 300, 30, 440, 166, 273, 292, 109, 65, 214, 2, 249, 258, 298, 454, 52, 420, 171, 104, 462, 100, 60, 393, 29, 64, 266, 340, 278, 480, 25, 144, 478, 39, 270, 106, 87, 164, 172, 177, 212, 121, 293, 55, 397, 345, 378, 69, 90, 264, 377, 453, 111, 416, 238, 325, 149, 263, 236, 9, 262, 296, 215, 20, 476, 385, 367, 230, 309, 290, 356, 225, 403, 182, 47, 472, 235, 276, 145, 105, 359, 481, 240, 181, 286, 314, 470, 441, 56, 449, 473, 248, 101, 139, 399, 304, 142, 430, 193, 208, 334, 170, 391, 3, 295, 88, 379, 250, 341, 28, 160, 155, 93, 118, 450, 91, 368, 117, 239, 324, 301, 360, 392, 267, 371, 8, 165, 22, 173, 223, 99, 123, 354, 31, 372, 19, 224, 187, 237, 488, 59, 246, 489, 425, 362, 274, 174, 386, 128, 200, 396, 451, 108, 471, 390, 131, 477, 492, 61, 247, 82, 259, 189]
    print('a:', a)
    b = []
    a_original = a[:]
    n = len(a)
    lis = [a[0], a[1]]
    print("LIS:", lis)
    contador = 0
    rotateLIS(lis)
    print("a:", a)
    print("b:", b)
    print("contador:", contador)
    while len(a) > 0:
        index_minimosPasos = calculaIndexPasosMinimos()
        giraPilas(a,b,index_minimosPasos)
        pb(a,b); contador += 1 # lo pasa de A a B haciendo pb
    situarMax_en_B()
    subirTodo_a_A()
    print("a:", a)
    print("b:", b)
    print(f"La pila A {'SI' if sorted(a_original)==a else 'No' } está ordenada.")
    print(f"contador: {contador}")

a: [161, 97, 226, 102, 11, 288, 148, 328, 154, 463, 73, 323, 284, 67, 34, 370, 228, 70, 199, 130, 424, 307, 245, 185, 496, 303, 119, 327, 457, 141, 24, 348, 406, 302, 349, 1, 80, 72, 319, 257, 285, 494, 419, 150, 308, 256, 135, 44, 412, 369, 206, 113, 10, 497, 413, 467, 136, 191, 180, 265, 126, 198, 27, 417, 125, 452, 346, 83, 280, 153, 326, 320, 232, 151, 49, 380, 474, 294, 21, 495, 461, 272, 414, 335, 482, 317, 120, 442, 234, 71, 358, 112, 210, 217, 405, 227, 357, 394, 281, 146, 485, 500, 75, 322, 404, 209, 408, 124, 350, 183, 50, 26, 332, 243, 375, 255, 143, 251, 337, 443, 204, 46, 374, 157, 268, 205, 382, 439, 305, 409, 220, 115, 339, 53, 498, 77, 6, 347, 277, 435, 96, 122, 388, 448, 78, 162, 446, 138, 127, 287, 137, 253, 433, 221, 18, 366, 459, 487, 84, 114, 196, 192, 312, 95, 468, 14, 315, 63, 23, 279, 297, 186, 411, 429, 269, 460, 479, 343, 94, 163, 74, 133, 398, 43, 438, 89, 5, 445, 45, 436, 156, 464, 383, 81, 129, 455, 418, 76, 355, 12, 484, 85, 353, 289, 68, 195, 132, 271, 36